## I. Introduction



As the capital city of US, Washington, D.C.is a popular city among both international and domestic tourists. We can see a record that nearly 22 million domestic tourists visited Washington D.C. in 2019. Thus, the restaurant business is also becoming very competitive. There are more than 2000 restaurants in Washington, D.C., if we search on google. \
\
As an entrepreneur, if he or she wants to open an restaurant in Washington, D.C., where will be a recommended location? Is it a good choice if he or she opens the restaurant where already exists many restaurants? Or try to find a place which is far away from other restaurants? After the restaurant is opened, how to make it popular? \
\
In this project, we will get to know more about D.C. restaurants and try to answer these questions. Finally, we will be able to provide some recommendations for entrepreneurs who want to open restaurants in Washington, D.C..


## II. Data


I will mainly use Foursquare API and k-means clusters to help conduct analysis. \
\
I will find the highly rated D.C. restaurants and visualize the data on a map; create "clusters" of these restaurants to identfiy potential districts for entrepreneurs to choose from. Entrepreneurs can choose one from these business districts to locate his or her restaurant. \
\
After entrepreneurs decide the district, they can check customer(user) tips which are provided by Foursquare API. They can know more about other restaurants around this area. Then, entrepreneurs have a chance to provide better services based on those reviews.

## III. Methodology

I conducted this analysis based on DC restaurants' data. \
\
First, I used Foursquare API to explore DC's trending restaurants. I set the limit as 100 restaurants and the radius as 2000 meter for each restaurant from their given latitude and longitude informations.\
\
Then, I used Folium to bulld a map that can show these restaurants. At the same time, I used unsupervised learning K-means algorithm to cluster the given restaurants. K-Means algorithm is one of the most common cluster method of unsupervised learning. We have four clusters thatcan let entrepreneurs to choose from.\
\
Finally, I use Foursquare API again to get several customer reviews from each cluster.

### Details are shown below:

In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests
import pandas as pd
import numpy as np
import random

from IPython.display import Image 
from IPython.core.display import HTML 

from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /Users/khoaphan/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0



geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-2.0.0          | 63 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
S

In [3]:
CLIENT_ID = 'OVEHUFXFNWI5M01WBKLRFSWX5G4BFC2JIYCVZR55V3YD5SV5' # your Foursquare ID
CLIENT_SECRET = 'AGHD5GO2WGARUWVEWYVILTPVARH2JJBITQF50JBUZXDBFV14' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OVEHUFXFNWI5M01WBKLRFSWX5G4BFC2JIYCVZR55V3YD5SV5
CLIENT_SECRET:AGHD5GO2WGARUWVEWYVILTPVARH2JJBITQF50JBUZXDBFV14


### Get DC's location

In [4]:
address = 'Washington, D.C.'

geolocator = Nominatim(user_agent="foursquare-agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.8949855 -77.0365708


In [5]:
search_query = 'restaurant'
radius = 2000
print(search_query + ' .... OK!')

restaurant .... OK!


In [6]:
# define URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f04b4bca45aef3f3e79cb5b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Northwest Washington',
  'headerFullLocation': 'Northwest Washington, Washington',
  'headerLocationGranularity': 'neighborhood',
  'query': 'restaurant',
  'totalResults': 244,
  'suggestedBounds': {'ne': {'lat': 38.91298551800001,
    'lng': -77.01348657327377},
   'sw': {'lat': 38.87698548199998, 'lng': -77.05965502672625}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '44014894f964a5201d301fe3',
       'name': 'Old Ebbitt Grill',
       'location': {'address': '675 15th St NW',
        'crossStreet': 'at G St NW',
        'la

### Transform restaurants from a json file to a dataframe

In [7]:
venues = results['response']['groups'][0]['items']

df0= json_normalize(venues)
df=pd.DataFrame()
df['venue_ID']=df0['venue.id']
df['name']=df0['venue.name']
df['lat']=df0['venue.location.lat']
df['lng']=df0['venue.location.lng']
df.head()

,venue_ID,name,lat,lng
0,44014894f964a5201d301fe3,Old Ebbitt Grill,38.897954,-77.033270
1,54dd3570498e265fdbac9167,Which Wich,38.896024,-77.042158
2,52cf4a9a498e3779a30ff04c,"Joe's Seafood, Prime Steak & Stone Crab",38.899984,-77.033939
3,554984c2498eed234444931d,Mastro's Steakhouse,38.897679,-77.029937
4,4a566b45f964a5201eb51fe3,BLT Steak,38.901720,-77.037600


### Use Folium to build a map

In [8]:
map_dc = folium.Map(location=[latitude, longitude], zoom_start=13)

In [9]:
from sklearn.cluster import KMeans

### Use K-means to build clusters

In [10]:
k_means = KMeans(init = "k-means++", n_clusters = 4, n_init = 12)
df_array=np.array(df[['lat','lng']])
k_means.fit(df_array)
k_labels=pd.DataFrame(k_means.labels_)
k_labels.info()
df['cluster']=k_labels
df.groupby('cluster').count()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
0    100 non-null int32
dtypes: int32(1)
memory usage: 528.0 bytes


,venue_ID,name,lat,lng
cluster,,,,
0,30,30,30,30
1,30,30,30,30
2,16,16,16,16
3,24,24,24,24


### Show restaurants after being clustered

In [11]:
for lat, lng, name,venue_ID,cluster in zip(df['lat'], df['lng'], df['name'],df['venue_ID'],df['cluster']):
    
    if cluster == 0:
        label = '{}, {}'.format(name, venue_ID)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7,
            parse_html=False).add_to(map_dc)  
    if cluster == 1:
        label = '{}, {}'.format(name, venue_ID)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map_dc)  
    if cluster == 2:
        label = '{}, {}'.format(name, venue_ID)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='green',
            fill=True,
            fill_color='green',
            fill_opacity=0.7,
            parse_html=False).add_to(map_dc)  
    if cluster == 3:
        label = '{}, {}'.format(name, venue_ID)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='purple',
            fill=True,
            fill_color='purple',
            fill_opacity=0.7,
            parse_html=False).add_to(map_dc)  
map_dc

### Generate the recommended location for the four cluster using the mean of the locations

In [12]:
center_location=df.groupby('cluster').mean()
center_location

,lat,lng
cluster,,
0,38.897604,-77.022760
1,38.903761,-77.040642
2,38.901323,-77.046563
3,38.901709,-77.029803


### Add recommended location on the map

In [13]:
i=0
for lat, lng in zip(center_location['lat'],center_location['lng']):
    folium.Marker([lat,lng], popup='Recommend for Group '+ str(i)).add_to(map_dc)
    i=i+1
map_dc

### If choose cluster == 0 (blue), let us check tips of two nearby restaurants.

In [14]:
venue_id01 = '59e38decdd8442066c07ca85' #Naf Naf Middle Eastern Grill

limit = 3000 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id01, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results01 = requests.get(url).json()

tips01 = results01['response']['tips']['items']

tip01 = results01['response']['tips']['items'][0]
tip01.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [15]:
pd.set_option('display.max_colwidth', -1)

tips01_df = json_normalize(tips01) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips01_filtered = tips01_df.loc[:, filtered_columns]

# display tips
tips01_filtered

/Users/khoaphan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Very good falafel (even though not freshly fried) and basmati rice. Skip the meats, get the falafel. The harissa is very tasty as well.",0,0,5e0243d93f0c4f00075277b4,Abhishek,S,NaN,57852015


In [16]:
venue_id02 = '53fcb7c4498ef3ae54309d5f' #DC Pizza

limit = 3000 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id02, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results02 = requests.get(url).json()

tips02 = results02['response']['tips']['items']

tip02 = results02['response']['tips']['items'][0]
tip02.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [17]:
pd.set_option('display.max_colwidth', -1)

tips02_df = json_normalize(tips02) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips02_filtered = tips02_df.loc[:, filtered_columns]

# display tips
tips02_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Fresh baked pizza. Service is good and fast. A lot of options to built your own pizza and the price is very fair. Pepperoni tasteful as always. Pesto and light cheese awesome as well.,0,0,58850f0f8ae36367e2abfcc2,Fernando,K,NaN,148118244


### If choose cluster == 1 (red), let us check tips of two nearby restaurants.¶

In [18]:
venue_id11 = '4a6e4e04f964a52074d41fe3' #Jaleo

limit = 3000 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id11, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results11 = requests.get(url).json()

tips11 = results11['response']['tips']['items']

tip11 = results11['response']['tips']['items'][0]
tip11.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

In [19]:
pd.set_option('display.max_colwidth', -1)

tips11_df = json_normalize(tips11) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips11_filtered = tips11_df.loc[:, filtered_columns]

# display tips
tips11_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"You don't often get authentic Spanish Tapas, but this place gets it just right. Get the pollo, chorizo, cauliflower. Avoid gambas al ajillo, sour and not enough garlic. Follow me on IG @simplysavor",8,0,567f3de8498ed236fc9d9557,Riddhima,H,NaN,1077719


In [20]:
venue_id12 = '54a7101c498eb46943224c11' #China Chilcano

limit = 3000 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id12, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results12 = requests.get(url).json()

tips12 = results12['response']['tips']['items']

tip12 = results12['response']['tips']['items'][0]
tip12.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [21]:
pd.set_option('display.max_colwidth', -1)

tips12_df = json_normalize(tips12) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips12_filtered = tips12_df.loc[:, filtered_columns]

# display tips
tips12_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Everything was amazing! The eclectic blend of Chinese, Japanese and Peruvian foods took my tastebuds on a world tour. I recommend the aeropuerto fried rice, nobu usuzukuri ceviche & lamb potstickers",8,0,54deb553498ead4df70c5408,Bobby,NaN,NaN,9940438


### If choose cluster == 2 (green), let us check tips of two nearby restaurants.

In [22]:
venue_id21 = '52cf4a9a498e3779a30ff04c' #Joe's Seafood, Prime Steak & Stone Crab

limit = 3000 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id21, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results21 = requests.get(url).json()

tips21 = results21['response']['tips']['items']

tip21 = results21['response']['tips']['items'][0]
tip21.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'startAt', 'agreeCount', 'disagreeCount', 'todo', 'user'])

In [23]:
pd.set_option('display.max_colwidth', -1)

tips21_df = json_normalize(tips21) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips21_filtered = tips21_df.loc[:, filtered_columns]

# display tips
tips21_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Everyday from 2:30 PM to 6:30 PM, enjoy cocktail hour at Joe’s in the beautiful Banking Hall, featuring half-priced Oysters on the Half Shell, signature cocktails, wines by the glass and draft beers.",13,0,5930563f1de7656bc56bf823,"Joe's Seafood, Prime Steak & Stone Crab",NaN,NaN,78030258


In [24]:
venue_id22 = '54189378498e368e1cecff45' #&pizza

limit = 3000 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id22, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results22 = requests.get(url).json()

tips22 = results22['response']['tips']['items']

tip22 = results22['response']['tips']['items'][0]
tip22.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

In [25]:
pd.set_option('display.max_colwidth', -1)

tips22_df = json_normalize(tips22) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips22_filtered = tips22_df.loc[:, filtered_columns]

# display tips
tips22_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,Very good. Only here for the weekend and it is my second time eating here. Really good concept and fresh ingredients. Best part open until 4A! Will be back. Customer service could be better.,3,0,5be7662dbe7078002c369d63,Desmand,W,NaN,62624461


### If choose cluster == 3 (purple), let us check tips of two nearby restaurants.

In [26]:
venue_id31 = '575ae890498e04099a6d1c67' #RPM italian

limit = 3000 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id31, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results31 = requests.get(url).json()

tips31 = results31['response']['tips']['items']

tip31 = results31['response']['tips']['items'][0]
tip31.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'photo', 'photourl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [27]:
pd.set_option('display.max_colwidth', -1)

tips31_df = json_normalize(tips31) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips31_filtered = tips31_df.loc[:, filtered_columns]

# display tips
tips31_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,A must try to steak lovers!! Better than STK!!! Wow!!! Haha!,12,0,57a1da7f498e8b30055ffba3,براندا,NaN,NaN,152490229


In [28]:
venue_id32 = '5aa6f0eee075507441b7a499' #The Capital Burger

limit = 3000 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id32, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results32 = requests.get(url).json()

tips32 = results32['response']['tips']['items']

tip32 = results32['response']['tips']['items'][0]
tip32.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [29]:
pd.set_option('display.max_colwidth', -1)

tips32_df = json_normalize(tips32) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips32_filtered = tips32_df.loc[:, filtered_columns]

# display tips
tips32_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"You can’t go wrong with any of the burgers, and they cook them exactly as you ask. Solid drink selection too.",14,0,5ab03060dd84424cbe5beefe,Robert,T,NaN,3883748


## IV. Results


As show above, we visualized four areas(clusters). We use geographic average of each area to recommended the location for a new restaurant. Also, tips of two nearby restaurants are shown as well.

## V. Discussion


When checking the map, we can see that restaurants that sell similar kind of food will not open close to each other. From the customer tips, we can know that food must be fresh and tasty, and service should be fast and nice.
This solution is based purely on the geographic location, which is quite good in most cases. Since in the restaurant business, the right location is the most important decision to make. Nevertheless, if we could find more information on each cluster,it will improve the solution further.


## VI. Conclusion

Finally, entrepreneurs are able to get ideas of where to open in DC and how to face customers now. However, to build a successful restaurant, it's also important to determine a solid business plan. This will involve many considerations, such as target customers, competition, theme and staffing, to give the best chances for success. Thus, before entrepreneurs can make the final decission, they need to consider other additional factors like what mentioned before. \
At the same time, if the outcomes of the clustering, especially when it yields a result different ideas of a 'business circle' in DC, we need to do more research. For some reason, maybe the clustering is wrong. But more likely, the result may present a new business opportunity. If we can identify the reason that cause the clustering, it might be a valuable business insight that we can observe.